## Spark streaming test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit import *
from kafka import KafkaConsumer

subreddit = "technology"
print("imported modules")

imported modules


In [2]:
spark = SparkSession.builder.appName("reddit_" + subreddit) \
            .master("spark://spark-master:7077") \
            .config("spark.eventLog.enabled", "true") \
            .config("spark.eventLog.dir", "file:///opt/workspace/events") \
            .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
            .getOrCreate()

spark.sparkContext.setLogLevel('WARN')
print("created spark successfully")

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2083e5f9-136a-40ae-9d67-a85f7357e5df;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

created spark successfully


In [4]:
test_df = spark \
            .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "kafka:9092") \
                .option("subscribe", "reddit_" + subreddit) \
                .option("startingOffsets", "earliest") \
                .load() \
                .selectExpr("CAST(value AS STRING)")

test_df.writeStream \
    .trigger(processingTime='5 seconds') \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "true") \
    .start() \
    .awaitTermination()

-------------------------------------------
Batch: 12
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



-------------------------------------------
Batch: 13
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|{"approved_at_utc...|
+--------------------+



In [16]:
consumer = KafkaConsumer(
    "reddit_" + subreddit,
     bootstrap_servers=['kafka:9092'],
     auto_offset_reset='latest',
     value_deserializer=lambda x: json.loads(x.decode('utf-8')))

for message in consumer:
    print(message.value)
    print("--------------------------------------")

KeyboardInterrupt: 